In [2]:
import gym
from env_batch import ParallelEnvBatch


def make_env():
    return gym.make('Pendulum-v0')


env = ParallelEnvBatch(make_env, nenvs=16)

print(env.observation_space.shape)
print(env.action_space.shape)

(3,)
(1,)


In [3]:
curr = env.reset()
print(curr.shape)

import numpy as np

obs = np.array([curr for _ in range(10)])

(16, 3)


In [1]:
from rlkits.policies import PolicyWithValue
from rlkits.env_batch import ParallelEnvBatch
import numpy as np
import gym
from rlkits.sampler import TrajectorySampler

def make_env():
    return gym.make('Pendulum-v0')

env = ParallelEnvBatch(make_env, nenvs=16)

#env = make_env()


ob_space = env.observation_space
ac_space = env.action_space

pi = PolicyWithValue(
    ob_space=ob_space, ac_space=ac_space, ckpt_dir='/tmp', 
    hidden_layers=[1024])


samp = TrajectorySampler(env, pi, 5)
#ac, log_prob, v = policy.step(curr)

Not a Vector Env
<class 'rlkits.env_batch.ParallelEnvBatch'>


In [5]:
if not isinstance(env, ParallelEnvBatch):
    print('false')

In [18]:
nx = np.random.rand(16, 3)

t = np.random.rand(16)
done = (t > 0.5)

In [21]:
done.shape

(16,)

In [20]:
nx[done]

array([[0.92563383, 0.55998717, 0.65613426],
       [0.94821461, 0.88353904, 0.36528434],
       [0.15110719, 0.7119068 , 0.7472019 ],
       [0.75858051, 0.70755973, 0.4250092 ],
       [0.69849213, 0.30402498, 0.11032107],
       [0.14048728, 0.29181518, 0.38692375],
       [0.79454275, 0.70728973, 0.29947912]])

In [ ]:
nx_state (2, 3)
rew (2,)
done (2,) bool

In [30]:
nxstate = np.array([
    [ 0.97061545, -0.24063592, -0.565105],
 [-0.18096242,0.98349001,2.07499883]])
done = np.array([False,False])

nxstate[~done]

array([[ 0.97061545, -0.24063592, -0.565105  ],
       [-0.18096242,  0.98349001,  2.07499883]])

In [24]:
nx_state = np.random.rand(2, 3)
done = np.array([False, False])

nx_state[~done]

array([[0.76228153, 0.38908259, 0.09040862],
       [0.76929873, 0.14689109, 0.05569494]])

# how to compute average episode rewards for a parallel env?
```
curr_ep_ret = [e1, e2, ..., en]

if env 2 is done
ep_ret[2].append(curr_ep_ret[2])
curr_ep_ret[2] = 0

```
so I get a list of list for `ep_rets` such that `ep_rets[i]` is the return from env i

Same idea can be applied to episodic length. Create a list of list for the 
`ep_lens` variable
such that `ep_lens[i]` is the length of episode for env i. 

In [1]:


from rlkits.policies import PolicyWithValue
from rlkits.env_batch import ParallelEnvBatch
from rlkits.sampler import ParallelEnvTrajectorySampler
from rlkits.env_wrappers import AutoReset, StartWithRandomActions
from rlkits.sampler import estimate_Q
import numpy as np
import gym
import pprint as pp


def make_env():
    env = gym.make('CartPole-v0').unwrapped
    env = AutoReset(env)
    env = StartWithRandomActions(env, max_random_actions=5)
    return env

def reward_transform(rew):
    return (rew + 8.0) / 16.0

env = ParallelEnvBatch(make_env, nenvs=4)

#env = make_env()
ob_space = env.observation_space
ac_space = env.action_space

pi = PolicyWithValue(
    ob_space=ob_space, ac_space=ac_space, ckpt_dir='/tmp', 
    hidden_layers=[1024])


samp = ParallelEnvTrajectorySampler(env, pi, 32)

for attr in ['obs', 'rews', 'vpreds', 'dones', 'actions', 'log_prob']:
    print(attr, getattr(samp, attr).shape)
traj = samp(callback=estimate_Q)
#pp.pprint(traj)

env.close()

Type of the environment <class 'rlkits.env_batch.ParallelEnvBatch'>
obs (32, 4, 4)
rews (32, 4)
vpreds (32, 4)
dones (32, 4)
actions (32, 4)
log_prob (32, 4)
[-0.6999495  -0.68569475 -0.68755704 -0.70016044]
[-0.6928578  -0.7072819  -0.6811818  -0.69384295]
[-0.6864343  -0.6854354  -0.7117607  -0.69867474]
[-0.7063655  -0.7076565  -0.68119943 -0.69432425]
[-0.69983524 -0.68494815 -0.7118937  -0.69821686]
[-0.69346255 -0.6782522  -0.7055345  -0.6915375 ]
[-0.6872151  -0.67161506 -0.686969   -0.70138544]
[-0.6810609 -0.722099  -0.6802192 -0.6912386]
[-0.67496914 -0.71603173 -0.6735384  -0.701736  ]
[-0.7179864  -0.71022016 -0.7201027  -0.69083756]
[-0.67536604 -0.7046337  -0.67266273 -0.6841817 ]
[-0.7177896  -0.699241   -0.66582555 -0.67761004]
[-0.67533934 -0.6940109  -0.72849035 -0.71569854]
[-0.6688719  -0.6974     -0.6925171  -0.67711085]
[-0.662423   -0.69647515 -0.6858849  -0.67042214]
[-0.7317598  -0.69360644 -0.6793447  -0.7234196 ]
[-0.7256433  -0.6989164  -0.67286634 -0.669432

In [2]:
pp.pprint(traj)

{'Q': array([[27.531776  , 27.482914  , 27.532213  , 27.5153    ],
       [26.799774  , 26.750418  , 26.800215  , 26.783133  ],
       [26.060377  , 26.010523  , 26.060823  , 26.043568  ],
       [25.313513  , 25.263155  , 25.313963  , 25.296534  ],
       [24.559103  , 24.508238  , 24.559559  , 24.541952  ],
       [23.797073  , 23.745695  , 23.797535  , 23.779749  ],
       [23.027348  , 22.975449  , 23.027813  , 23.009848  ],
       [22.249846  , 22.197422  , 22.250317  , 22.232168  ],
       [21.46449   , 21.411537  , 21.464966  , 21.446634  ],
       [20.671204  , 20.617714  , 20.671682  , 20.653166  ],
       [19.869902  , 19.815872  , 19.870386  , 19.851683  ],
       [19.060507  , 19.005932  , 19.060995  , 19.042103  ],
       [18.242935  , 18.187809  , 18.24343   , 18.224346  ],
       [17.417107  , 17.361423  , 17.417604  , 17.398329  ],
       [16.582935  , 16.52669   , 16.583439  , 16.563969  ],
       [15.740337  , 15.683525  , 15.740849  , 15.721181  ],
       [14.88923  

In [21]:
traj = samp(callback=estimate_Q)
pp.pprint(traj)

{'Q': array([[27.498121 , 27.498745 , 27.501127 , 27.501968 ],
       [26.76578  , 26.766409 , 26.768816 , 26.769665 ],
       [26.02604  , 26.026676 , 26.029106 , 26.029964 ],
       [25.278828 , 25.27947  , 25.281925 , 25.282793 ],
       [24.524069 , 24.524717 , 24.527197 , 24.528074 ],
       [23.761686 , 23.76234  , 23.764845 , 23.765732 ],
       [22.991602 , 22.992264 , 22.994793 , 22.99569  ],
       [22.21374  , 22.214407 , 22.216963 , 22.217867 ],
       [21.428019 , 21.428694 , 21.431276 , 21.432188 ],
       [20.634361 , 20.635044 , 20.637653 , 20.638573 ],
       [19.832687 , 19.833378 , 19.836014 , 19.836943 ],
       [19.022917 , 19.023613 , 19.026276 , 19.027214 ],
       [18.204966 , 18.20567  , 18.208359 , 18.209307 ],
       [17.378754 , 17.379463 , 17.382181 , 17.383139 ],
       [16.544195 , 16.544912 , 16.547657 , 16.548624 ],
       [15.701207 , 15.701932 , 15.704704 , 15.705681 ],
       [14.849704 , 14.850436 , 14.853236 , 14.854223 ],
       [13.989599 , 13.99

In [20]:
import torch
x = traj['obs']
x = pi.transform_input(x)

with torch.no_grad():
    y = pi.policy_net(x)
    dist = pi.dist(y)
    action = dist.sample()
    log_prob = dist.log_prob(action)
    print(log_prob)

tensor([[-0.6939, -0.6865, -0.6912, -0.6926],
        [-0.6935, -0.6992, -0.6963, -0.6927],
        [-0.6939, -0.6938, -0.6909, -0.6938],
        [-0.6912, -0.6949, -0.6920, -0.6912],
        [-0.6922, -0.6901, -0.6931, -0.6899],
        [-0.6953, -0.6911, -0.6942, -0.6955],
        [-0.6966, -0.6941, -0.6931, -0.6945],
        [-0.6884, -0.6910, -0.6942, -0.6905],
        [-0.6870, -0.6943, -0.6909, -0.6949],
        [-0.6877, -0.6931, -0.6897, -0.6939],
        [-0.6885, -0.6943, -0.6957, -0.6911],
        [-0.6970, -0.6931, -0.6916, -0.6921],
        [-0.6880, -0.6920, -0.6936, -0.6908],
        [-0.6865, -0.6908, -0.6915, -0.6968],
        [-0.6932, -0.6945, -0.6960, -0.6960],
        [-0.6942, -0.6934, -0.6913, -0.6973],
        [-0.6931, -0.6945, -0.6924, -0.6965],
        [-0.6943, -0.6957, -0.6928, -0.6933],
        [-0.6930, -0.6947, -0.6924, -0.6922],
        [-0.6944, -0.6959, -0.6951, -0.6953],
        [-0.6957, -0.6914, -0.6899, -0.6966],
        [-0.6894, -0.6902, -0.6954

In [9]:
print(y)

tensor([[[-7.1273e-04,  8.1827e-04],
         [-8.1811e-03, -2.1599e-02],
         [-1.0436e-03, -4.9530e-03],
         [ 2.1454e-03,  3.3230e-03]],

        [[-3.1625e-03, -3.8199e-03],
         [-5.3431e-03, -1.7407e-02],
         [-3.3457e-03, -9.7168e-03],
         [-3.2782e-04, -1.2684e-03]],

        [[-7.6959e-04,  8.1444e-04],
         [-1.5940e-04,  1.1194e-03],
         [-7.9231e-04, -5.2185e-03],
         [ 2.0364e-03,  3.4183e-03]],

        [[ 1.5648e-03,  5.5260e-03],
         [ 2.2190e-03,  5.8093e-03],
         [ 1.7130e-03, -6.4688e-04],
         [ 4.3392e-03,  8.1836e-03]],

        [[-9.5156e-04,  9.6948e-04],
         [ 4.5377e-03,  1.0577e-02],
         [ 4.1665e-03,  4.0002e-03],
         [ 6.5761e-03,  1.3029e-02]],

        [[ 1.3696e-03,  5.6880e-03],
         [ 2.0029e-03,  6.0811e-03],
         [ 6.5643e-03,  8.7250e-03],
         [ 3.9463e-03,  8.6272e-03]],

        [[ 3.6271e-03,  1.0485e-02],
         [-4.8248e-04,  1.5113e-03],
         [ 4.1123e-03,  4.

In [16]:
dist = pi.dist(y[0])

In [17]:
actions = dist.sample()

In [18]:
dist.log_prob(actions)

tensor([-0.6924, -0.6999, -0.6951, -0.6937], grad_fn=<SqueezeBackward1>)